In [6]:
import pandas as pd
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
import re

# **Device Configuration**
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")

# **Email Body Cleaning Function**
def clean_email_body(body):
    # Remove quoted replies (lines starting with ">")
    body = re.sub(r"(^|\n)>.*", "", body)
    # Remove excessive asterisks and separator lines
    body = re.sub(r"\*{2,}.*", "", body)
    # Remove trailing email artifacts and forward markers
    body = re.sub(r"(\n-----.*|----- Forwarded by.*|----------------------.*)", "", body)
    # Remove email signatures and disclaimers
    body = re.sub(r"(--\\s*|\\nRegards,|\\nBest,|\\nSincerely,).*", "", body, flags=re.IGNORECASE)
    body = re.sub(r"(Disclaimer:|This e-mail is the property of).*", "", body, flags=re.IGNORECASE)
    # Replace multiple newlines with a single space
    body = re.sub(r"\s+", " ", body).strip()
    return body
    
# **Simplify Metadata for Each Email**
def simplify_metadata(row):
    # Fix issues with list parsing in "To"
    recipients = ", ".join(row["to"]) if isinstance(row["to"], list) else row["to"]
    return f"From: {row['from']} To: {recipients} Subject: {row['subject']} Body: {clean_email_body(row['body'])}"

# **Preprocessing Function**
def preprocess_threads_optimized(df):
    grouped = df.groupby("thread_id").apply(lambda x: {
        "input_text": " ".join(
            pd.Series([
                simplify_metadata(row)  # Format each email
                for _, row in x.iterrows()
            ]).drop_duplicates().tolist()
        ),
        "summary": x["summary"].iloc[0]
    }).reset_index(drop=True)
    return pd.DataFrame(grouped.tolist())

# **Load Dataset**
from datasets import load_dataset
ds = load_dataset("xprilion/email-summary-dataset")
df = pd.DataFrame(ds["train"])

# **Apply Preprocessing**
preprocessed_data = preprocess_threads_optimized(df)

Using device: cuda


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

data.csv:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21684 [00:00<?, ? examples/s]

In [7]:
# **Validate Cleaning and Formatting**
print("Sample processed input_text:")
print(preprocessed_data["input_text"].sample(2, random_state=42).values)

Sample processed input_text:
['From: Eric Bass To: [\'Phillip M Love\'] Subject: Trade Body: I offered you my 1 and 3 for your 1 and 4. I can throw in my 5 for your 6. Let me know. Eric From: Eric Bass To: [\'Jason.Bass2@COMPAQ.com\'] Subject: Trade Body: The 24th (2) pick and the 42nd (4) pick for your 2nd round and your 7th round? From: Eric Bass To: [\'lqcolombo@aol.com\'] Subject: Trade Body: Muhsin Muhammed and Elvis Grbac for Jeff Garcia, Raymont Harris, and Wayne Chrebet? Gives you starting QB and depth at RB (all 4 of your RBs will likely not play week 1 and maybe 2). Let me know From: Eric Bass To: [\'Jason.Bass2@COMPAQ.com\'] Subject: Trade Body: Hear anything on Engram? From: Eric Bass To: ["O\'Neal D Winfree"] Subject: Trade Body: Are you going to put it in the system or do you just want me to do it? From: Eric Bass To: [\'Steve Venturatos\'] Subject: Trade Body: Are we done on that trade? Richardson, Bettis, and Chrebet for Freeman 3 Starters for 1 From: Eric Bass To: [\'S

In [8]:
# Split data
train_size = int(0.8 * len(preprocessed_data))
train_data = preprocessed_data[:train_size]
test_data = preprocessed_data[train_size:]

# Initialize tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
# Dataset Class
class EmailDataset(Dataset):
    def __init__(self, data, tokenizer, max_input_length=1024, max_summary_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_summary_length = max_summary_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Get single example
        item = self.data.iloc[idx]
        
        # Tokenize input and summary on-the-fly
        inputs = self.tokenizer(
            item['input_text'],
            max_length=self.max_input_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        
        labels = self.tokenizer(
            item['summary'],
            max_length=self.max_summary_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        
        # Remove batch dimension added by tokenizer
        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'labels': labels['input_ids'].squeeze(0)
        }
    
    @staticmethod
    def collate_fn(batch):
        # Combine batch elements efficiently
        batch_inputs = {
            'input_ids': torch.stack([x['input_ids'] for x in batch]),
            'attention_mask': torch.stack([x['attention_mask'] for x in batch]),
            'labels': torch.stack([x['labels'] for x in batch])
        }
        return batch_inputs

# Create datasets
train_dataset = EmailDataset(train_data, tokenizer)
test_dataset = EmailDataset(test_data, tokenizer)

In [5]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import os

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Ensure GPU 
is used efficiently
torch.cuda.empty_cache()

# Create dataloaders
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=8, 
    shuffle=True,
    collate_fn=EmailDataset.collate_fn
)

test_dataloader = DataLoader(
    test_dataset, 
    batch_size=8,
    collate_fn=EmailDataset.collate_fn
)

# Model and Optimizer
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = torch.nn.DataParallel(model)
model.to(device)

# Initialize optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
lr_scheduler = StepLR(optimizer, step_size=1, gamma=0.9)

# Early Stopping Parameters
patience = 2
best_loss = float("inf")
epochs_without_improvement = 0

# Training Loop
num_epochs = 10
gradient_accumulation_steps = 4

# Create directory for saving model
os.makedirs("./saved_model", exist_ok=True)

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    num_batches = 0
    
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    
    # Initialize progress bar
    progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}")
    
    for step, batch in enumerate(progress_bar):
        # Move batch to device
        batch = {key: value.to(device) for key, value in batch.items()}
        
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"]
        )
        
        # Compute loss
        loss = outputs.loss
        if loss.ndimension() > 0:
            loss = loss.mean()
        
        # Gradient accumulation
        loss = loss / gradient_accumulation_steps
        loss.backward()
        
        if (step + 1) % gradient_accumulation_steps == 0:
            # Gradient clipping (optional)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            optimizer.zero_grad()
        
        # Update progress bar
        progress_bar.set_postfix({"loss": loss.item() * gradient_accumulation_steps})
        
        # Accumulate loss
        epoch_loss += loss.item() * gradient_accumulation_steps
        num_batches += 1
    
    # Step the learning rate scheduler once per epoch
    lr_scheduler.step()
    
    # Calculate average loss for the epoch
    avg_epoch_loss = epoch_loss / num_batches
    print(f"\nEpoch {epoch + 1} Average Loss: {avg_epoch_loss:.4f}")
    
    # Early stopping check
    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss
        epochs_without_improvement = 0
        print(f"New best loss achieved: {best_loss:.4f}")
        
        # Save the best model
        try:
            if isinstance(model, torch.nn.DataParallel):
                model.module.save_pretrained("./saved_model")
            else:
                model.save_pretrained("./saved_model")
            tokenizer.save_pretrained("./saved_model")
            print("Model saved successfully")
        except Exception as e:
            print(f"Error saving model: {e}")
    else:
        epochs_without_improvement += 1
        
    if epochs_without_improvement >= patience:
        print(f"Early stopping triggered at epoch {epoch + 1}")
        break

print("\nTraining completed!")

Using device: cuda


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Using 2 GPUs!

Epoch 1/10


Training Epoch 1:   0%|          | 0/417 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Training Epoch 1: 100%|██████████| 417/417 [16:22<00:00,  2.36s/it, loss=1.19] 


...


Epoch 8 Average Loss: 0.7284
New best loss achieved: 0.7284
Model saved successfully

Epoch 9/10


Training Epoch 9:   0%|          | 0/417 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Training Epoch 9: 100%|██████████| 417/417 [16:23<00:00,  2.36s/it, loss=1.2]  
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}



Epoch 9 Average Loss: 0.6777
New best loss achieved: 0.6777
Model saved successfully

Epoch 10/10


Training Epoch 10:   0%|          | 0/417 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Training Epoch 10: 100%|██████████| 417/417 [16:23<00:00,  2.36s/it, loss=0.617]
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}



Epoch 10 Average Loss: 0.6324
New best loss achieved: 0.6324
Model saved successfully

Training completed!


In [6]:
from transformers import BartForConditionalGeneration, BartTokenizer, GenerationConfig
import torch

# Load the saved model and tokenizer
model_path = "./saved_model"

model = BartForConditionalGeneration.from_pretrained(model_path)
tokenizer = BartTokenizer.from_pretrained(model_path)

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load generation config if previously saved
try:
    generation_config = GenerationConfig.from_pretrained(model_path)
except:
    # Default generation config if not saved
    generation_config = GenerationConfig(
        max_length=142,
        min_length=56,
        early_stopping=True,
        num_beams=4,
        length_penalty=2.0,
        no_repeat_ngram_size=3
    )

# Function to generate a summary
def generate_summary(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="longest").to(device)

    # Generate summary
    with torch.no_grad():
        summary_ids = model.generate(
            inputs["input_ids"],
            generation_config=generation_config
        )

    # Decode the generated summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [7]:
# Example text
example_text = """
Subject: Mandatory Participation Requirements for NSO Credits
    From: NSO Secretary
    To: Students, Mahavir, DOSA
    
    Body:
    Dear Students,

Welcome to the NSO program for this semester. Please note the inclusion of a mandatory component—the Run for Wellbeing (R4W) Event—as part of the credit requirements. Below are the detailed guidelines:

Key Requirements for NSO Credits
For students registering for 2 credits (28 hours):
10 hours must be completed through participation in 10 R4W sessions (1 session = 1 hour).
Each R4W session involves a 5 km run or walk.
Attendance in a minimum of 10 sessions is mandatory.
The remaining 18 hours must be completed through other sports activities organized during the semester.

Important Grading Guidelines:

Completing 10 R4W + 18 sports = 28 hours will guarantee ‘Satisfactory’ S grades for both credits (Grade: SS).
Failing to complete 10 R4W sessions will result in at least one ‘Unsatisfactory’ X grade (Grades: XS/SX/XX).
Failing to complete 5 R4W sessions will result in ‘Unsatisfactory’ X grades in both credits (Grade: XX).
For students registering for 1 credit (14 hours):
5 hours must be completed through 5 R4W sessions.
The remaining 9 hours must be covered through other sports activities to guarantee an S grade.
R4W Calendar

The schedule of R4W sessions is attached to this email. Ensure you plan your participation accordingly to avoid any last-minute challenges.

Tracking and Attendance
Track your activity using the Strava app.
Download link: Strava App
Take a screenshot of your completed activity on Strava, displaying the distance and time.
Submit your screenshot via a Google form that will be shared after the event to mark your attendance.

For any queries or clarifications, please feel free to contact us.

Best regards,



--



AKSHAT KUMAR AND BUBLI BRAHMA  

NSO Secretaries
Indian Institute of Technology, Bhilai     
Contact No. 7737288510, 6002846132
"""

# Generate and print the summary
summary = generate_summary(example_text)
print("Generated Summary:")
print(summary)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Generated Summary:
The Run for Wellbeing (R4W) event is a mandatory component of the NSO program for students registering for two credits (28 hours). Each R4W session involves a 5 km run or walk, and attendance in a minimum of 10 sessions is mandatory. The remaining 18 hours must be completed through other sports activities during the semester. Failing to complete these sessions will result in unsatisfactory X grades in both credits. Tracking and attendance are provided, and students can also use the Strava app to track their activity and take a screenshot to mark their attendance.


In [10]:
!pip install rouge

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [13]:
from rouge import Rouge

# Load model and tokenizer
model_path = "/kaggle/working/saved_model/"  # Path to your saved model
model = BartForConditionalGeneration.from_pretrained(model_path).to("cuda")
tokenizer = BartTokenizer.from_pretrained(model_path)

# Create DataLoader for test dataset
test_dataloader = DataLoader(
    test_dataset,
    batch_size=8,  # Adjust batch size based on GPU memory
    collate_fn=EmailDataset.collate_fn,
    shuffle=False
)

# Function to generate summaries
def generate_summary_from_dataloader(dataloader, model, tokenizer, device="cuda"):
    model.eval()
    generated_summaries = []
    reference_summaries = []

    with torch.no_grad():
        for batch in dataloader:
            # Move batch to device
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            
            # Generate summaries
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=128,  # Adjust max_length as needed
                min_length=30,
                num_beams=4,
                length_penalty=2.0
            )
            
            # Decode generated summaries and references
            generated_summaries.extend(
                tokenizer.batch_decode(outputs, skip_special_tokens=True)
            )
            reference_summaries.extend(
                tokenizer.batch_decode(labels, skip_special_tokens=True)
            )
    
    return generated_summaries, reference_summaries

# Generate and evaluate summaries
generated_summaries, reference_summaries = generate_summary_from_dataloader(
    test_dataloader, model, tokenizer
)

# Calculate ROUGE scores
rouge = Rouge()
scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)

# Print the ROUGE scores
print("ROUGE-1:", scores["rouge-1"])
print("ROUGE-2:", scores["rouge-2"])
print("ROUGE-L:", scores["rouge-l"])


ROUGE-1: {'r': 0.4973023626945872, 'p': 0.4503773073674005, 'f': 0.46858544538854363}
ROUGE-2: {'r': 0.24413624391534525, 'p': 0.21660355647022325, 'f': 0.2271890926861715}
ROUGE-L: {'r': 0.46844449909896385, 'p': 0.4245539374534186, 'f': 0.44157005926890974}


In [5]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer

# Load your fine-tuned BART model and tokenizer
model_name = "/kaggle/working/saved_model/"  # Replace with your model path
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

def generate_streaming(text, max_length=142, min_length=56, num_beams=4, length_penalty=2.0, no_repeat_ngram_size=3):
    # Tokenize the input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

    

    # Generate token IDs incrementally
    generated_ids = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        min_length=min_length,
        early_stopping=True,
        num_beams=num_beams,
        length_penalty=length_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        output_scores=False,  # Scores are not needed for streaming
        return_dict_in_generate=True,
    )

    # Decode and stream tokens
    generated_tokens = []
    for token_id in generated_ids.sequences[0]:
        generated_tokens.append(token_id.item())  # Convert token ID to Python int
        current_output = tokenizer.decode(generated_tokens, skip_special_tokens=True)
        print(current_output, end="\r", flush=True)  # Display the current output incrementally
        yield current_output  # Optionally send the output to an external UI/frontend

# Example text
example_text = """
Subject: Mandatory Participation Requirements for NSO Credits
From: NSO Secretary
To: Students, Mahavir, DOSA

Body:
Dear Students,

Welcome to the NSO program for this semester. Please note the inclusion of a mandatory component—the Run for Wellbeing (R4W) Event—as part of the credit requirements. Below are the detailed guidelines:

Key Requirements for NSO Credits:
1. For students registering for 2 credits (28 hours):
   - 10 hours must be completed through participation in 10 R4W sessions (1 session = 1 hour).
   - Each R4W session involves a 5 km run or walk.
   - Attendance in a minimum of 10 sessions is mandatory.
   - The remaining 18 hours must be completed through other sports activities organized during the semester.

2. Important Grading Guidelines:
   - Completing 10 R4W + 18 sports = 28 hours will guarantee ‘Satisfactory’ S grades for both credits (Grade: SS).
   - Failing to complete 10 R4W sessions will result in at least one ‘Unsatisfactory’ X grade (Grades: XS/SX/XX).
   - Failing to complete 5 R4W sessions will result in ‘Unsatisfactory’ X grades in both credits (Grade: XX).

3. For students registering for 1 credit (14 hours):
   - 5 hours must be completed through 5 R4W sessions.
   - The remaining 9 hours must be covered through other sports activities to guarantee an S grade.

R4W Calendar:
The schedule of R4W sessions is attached to this email. Ensure you plan your participation accordingly to avoid any last-minute challenges.

Tracking and Attendance:
- Track your activity using the Strava app. Download link: [Strava App]
- Take a screenshot of your completed activity on Strava, displaying the distance and time.
- Submit your screenshot via a Google form that will be shared after the event to mark your attendance.

For any queries or clarifications, please feel free to contact us.

Best regards,
AKSHAT KUMAR AND BUBLI BRAHMA  
NSO Secretaries  
Indian Institute of Technology, Bhilai  
Contact No. 7737288510, 6002846132
"""

# Generate and stream the summary
print("Generated Summary (streamed):")
for partial_summary in generate_streaming(example_text):
    pass  # The output is printed incrementally


Generated Summary (streamed):
